In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.rcParams['figure.figsize'] = (16,8)

In [2]:
train = pd.read_csv('../input/train.csv', index_col='PassengerId')
infer = pd.read_csv('../input/test.csv', index_col='PassengerId')

In [3]:
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
infer.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
full = pd.concat([train, infer])

C:\Users\vasil\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [6]:
full['Age'].fillna(-1, inplace=True)

In [7]:
train = full[full['Survived'].isnull() == False]
infer = full[full['Survived'].isnull() == True]

In [8]:
def split(df):
    from sklearn.model_selection import train_test_split
    
    X = df.drop('Survived', axis=1)
    y = df['Survived']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=379582)
    
    return pd.concat([X_train, y_train], axis=1), pd.concat([X_test, y_test],axis=1)

In [9]:
train, test = split(train)

In [10]:
import tensorflow as tf

In [16]:
# read feature columns from df
sex = tf.feature_column.categorical_column_with_vocabulary_list('Sex', vocabulary_list=['male', 'female'])
pclass = tf.feature_column.categorical_column_with_identity('Pclass', 4)
parch = tf.feature_column.categorical_column_with_identity('Parch', 10)
sibsp = tf.feature_column.categorical_column_with_identity('SibSp', 9)
age = tf.feature_column.numeric_column('Age')

# bucketizing age
age_b = tf.feature_column.bucketized_column(age,[0,7,12,18,30,36,44,55,65])

# defining feature crosses
parch_sib = tf.feature_column.crossed_column([parch,sibsp], 10 * 9)
age_class = tf.feature_column.crossed_column([age_b, pclass], 10 * 4)
age_sex = tf.feature_column.crossed_column([age_b, sex], 10 * 2)
sex_class = tf.feature_column.crossed_column([sex, pclass], 2 * 4)
parch_sib_age = tf.feature_column.crossed_column([parch,sibsp,age_b], 10 * 9 * 10)

# linear model
wide_columns = [
    sex, pclass, parch, sibsp,
    
    age_b,
    
    parch_sib, age_class, age_sex, sex_class, parch_sib_age
]

# neural network
deep_columns = [
    age_b,
    
    tf.feature_column.embedding_column(parch_sib, 5),
    tf.feature_column.embedding_column(age_class, 5),
    tf.feature_column.embedding_column(age_sex, 5),
    tf.feature_column.embedding_column(sex_class, 5),
    tf.feature_column.embedding_column(parch_sib_age, 10)
]

In [17]:
log = './DNN_CLASSIFIER'

config = tf.estimator.RunConfig(
    save_summary_steps=300, 
    save_checkpoints_secs=600, 
    keep_checkpoint_max=5, 
    log_step_count_steps=100, 
)

model = tf.estimator.DNNLinearCombinedClassifier(
    model_dir = log, 
    linear_feature_columns=wide_columns, 
    dnn_feature_columns=deep_columns, 
    dnn_hidden_units=[64,64,64,64], 
    n_classes=2, 
    dnn_dropout=0.2, 
    config=config, 
    dnn_optimizer=tf.train.AdamOptimizer(learning_rate=0.001)
)

INFO:tensorflow:Using config: {'_model_dir': './DNN_CLASSIFIER', '_tf_random_seed': None, '_save_summary_steps': 300, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000019FB9F03160>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [18]:
def train_input_fn(df, batch_size):
    return tf.estimator.inputs.pandas_input_fn(
        x = df[['Sex', 'Pclass', 'Parch', 'SibSp', 'Age']],
        y = df['Survived'],
        num_epochs = 1,
        shuffle = True,
        batch_size = batch_size)

def eval_input_fn(df):
    return tf.estimator.inputs.pandas_input_fn(
        x = df[['Sex', 'Pclass', 'Parch', 'SibSp', 'Age']],
        y = df['Survived'],
        num_epochs = 1,
        shuffle = False)

def pred_input_fn(df):
    return tf.estimator.inputs.pandas_input_fn(
        x = df[['Sex', 'Pclass', 'Parch', 'SibSp', 'Age']],
        num_epochs = 1,
        shuffle = False)

In [19]:
train_spec = tf.estimator.TrainSpec(input_fn = train_input_fn(train, batch_size=256))
eval_spec = tf.estimator.EvalSpec(input_fn = eval_input_fn(test))

In [27]:
for i in range(0,100):
    tf.estimator.train_and_evaluate(model, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CLASSIFIER\model.ckpt-300
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 300 into ./DNN_CLASSIFIER\model.ckpt.
INFO:tensorflow:loss = 86.19072, step = 300
INFO:tensorflow:Saving checkpoints for 303 into ./DNN_CLASSIFIER\model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-16-20:51:04
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring pa

INFO:tensorflow:Saving checkpoints for 312 into ./DNN_CLASSIFIER\model.ckpt.
INFO:tensorflow:loss = 68.26219, step = 312
INFO:tensorflow:Saving checkpoints for 315 into ./DNN_CLASSIFIER\model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-16-20:51:54
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CLASSIFIER\model.ckpt-315
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-16-20:51:56
INFO:tensorflow:Saving dict for global step 315: accuracy = 0.7728813, accuracy_baseline = 0.61355937, auc = 0.8346418, auc_precision_recall = 0.8030453, average_loss = 0.54942536, global_step = 315, label/mean = 0.38644066, loss = 54.02683, precision = 0.742268, prediction/mean = 0.3867103, recall = 0.6315789
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 315: ./DNN_CLASSIFIER\model.ckpt-315
INF

INFO:tensorflow:Restoring parameters from ./DNN_CLASSIFIER\model.ckpt-327
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-16-20:52:57
INFO:tensorflow:Saving dict for global step 327: accuracy = 0.77627116, accuracy_baseline = 0.61355937, auc = 0.83612007, auc_precision_recall = 0.8015152, average_loss = 0.5481264, global_step = 327, label/mean = 0.38644066, loss = 53.899094, precision = 0.74, prediction/mean = 0.39990246, recall = 0.64912283
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 327: ./DNN_CLASSIFIER\model.ckpt-327
INFO:tensorflow:Loss for final step: 26.396233.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling mode

INFO:tensorflow:Loss for final step: 19.317167.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CLASSIFIER\model.ckpt-339
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 339 into ./DNN_CLASSIFIER\model.ckpt.
INFO:tensorflow:loss = 67.82852, step = 339
INFO:tensorflow:Saving checkpoints for 342 into ./DNN_CLASSIFIER\model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-16-20:53:58
INFO:tensorflow:G

INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 351 into ./DNN_CLASSIFIER\model.ckpt.
INFO:tensorflow:loss = 70.27105, step = 351
INFO:tensorflow:Saving checkpoints for 354 into ./DNN_CLASSIFIER\model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-16-20:54:50
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CLASSIFIER\model.ckpt-354
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-16-20:54:52
INFO:tensorflow:Saving dict for global step 354: accuracy = 0.76949155, accuracy_baseline = 0.61355937, auc = 0.83764666, auc_precision_recall = 0.80363965, average_loss = 0.54727685, global_step = 354, label/mean = 0.38644066, loss = 53.815556, precision = 0.7346939, prediction/mean = 0.39148504, recall = 0.6315789
INFO:tensorflow:Saving 'checkpoint_path' summary for gl

...

INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 546 into ./DNN_CLASSIFIER\model.ckpt.
INFO:tensorflow:loss = 83.43729, step = 546
INFO:tensorflow:Saving checkpoints for 549 into ./DNN_CLASSIFIER\model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-16-21:09:55
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CLASSIFIER\model.ckpt-549
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-16-21:09:57
INFO:tensorflow:Saving dict for global step 549: accuracy = 0.7898305, accuracy_baseline = 0.61355937, auc = 0.83381796, auc_precision_recall = 0.80583817, average_loss = 0.61765254, global_step = 549, label/mean = 0.38644066, loss = 60.735836, precision = 0.7280702, prediction/mean = 0.41039065, recall = 0.7280702
INFO:tensorflow:Saving 'checkpoint_path' summary for glo

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CLASSIFIER\model.ckpt-561
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-16-21:10:50
INFO:tensorflow:Saving dict for global step 561: accuracy = 0.7661017, accuracy_baseline = 0.61355937, auc = 0.8328245, auc_precision_recall = 0.8032721, average_loss = 0.6236162, global_step = 561, label/mean = 0.38644066, loss = 61.32226, precision = 0.71428573, prediction/mean = 0.40266892, recall = 0.65789473
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 561: ./DNN_CLASSIFIER\model.ckpt-561
INFO:tensorflow:Loss for final step: 27.153954.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoint

INFO:tensorflow:Loss for final step: 18.550343.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CLASSIFIER\model.ckpt-573
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 573 into ./DNN_CLASSIFIER\model.ckpt.
INFO:tensorflow:loss = 89.841705, step = 573
INFO:tensorflow:Saving checkpoints for 576 into ./DNN_CLASSIFIER\model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-16-21:11:53
INFO:tensorflow:

INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 585 into ./DNN_CLASSIFIER\model.ckpt.
INFO:tensorflow:loss = 79.4045, step = 585
INFO:tensorflow:Saving checkpoints for 588 into ./DNN_CLASSIFIER\model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-16-21:13:00
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CLASSIFIER\model.ckpt-588
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-16-21:13:02
INFO:tensorflow:Saving dict for global step 588: accuracy = 0.76949155, accuracy_baseline = 0.61355937, auc = 0.83309096, auc_precision_recall = 0.80301654, average_loss = 0.6218956, global_step = 588, label/mean = 0.38644066, loss = 61.153072, precision = 0.73, prediction/mean = 0.40099943, recall = 0.6403509
INFO:tensorflow:Saving 'checkpoint_path' summary for global st

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CLASSIFIER\model.ckpt-600
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-16-21:13:55
INFO:tensorflow:Saving dict for global step 600: accuracy = 0.7627119, accuracy_baseline = 0.61355937, auc = 0.8310798, auc_precision_recall = 0.79739285, average_loss = 0.63906705, global_step = 600, label/mean = 0.38644066, loss = 62.841595, precision = 0.7075472, prediction/mean = 0.4135172, recall = 0.65789473
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 600: ./DNN_CLASSIFIER\model.ckpt-600
INFO:tensorflow:Loss for final step: 21.253475.


In [22]:
generator = model.predict(input_fn=pred_input_fn(infer), checkpoint_path='./DNN_CLASSIFIER/model.ckpt-288')

In [23]:
predictions = [next(generator) for i in range(len(infer))]
values = [val['class_ids'].tolist()[0] for val in predictions]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./DNN_CLASSIFIER/model.ckpt-288
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [24]:
submission = pd.DataFrame()

submission['PassengerId'] = infer.index
submission['Survived'] = values

submission.to_csv('../output/DNN_Linear_Combined_Classifier_V3.csv', index=False)